In [ ]:
#Name: Trevor Goad
#Class: ECGR 4106
#Assignment: HW 3
#Date: 3/30/2022
#github link:

In [5]:
from matplotlib import pyplot as plt
import numpy as np
import collections
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [7]:
from torchvision import datasets
from torchvision import transforms
data_path = './tmp/tensorflow/mnist/input_data'
cifar10 = datasets.CIFAR10(data_path, train = True, download = True,
                          transform=transforms.ToTensor())
cifar10_val = datasets.CIFAR10(data_path, train = False, download = True,
                              transform=transforms.ToTensor())

Files already downloaded and verified
Files already downloaded and verified


In [8]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16,8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.fc2 = nn.Linear(32, 10)
    
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8 * 8 * 8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [9]:
import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            
            outputs = model(imgs)
            
            loss = loss_fn(outputs, labels)
            
            optimizer.zero_grad()
            
            loss.backward()
            
            optimizer.step()
            
            loss_train += loss.item()
            
        if epoch == 1 or epoch % 10 ==0:
            print('{} Epoch {}, Training Loss {}'.format(datetime.datetime.now(),
                                                        epoch,
                                                        loss_train / len(train_loader)))

In [11]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                          shuffle=True)

model = Net()
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 300,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

2022-03-30 16:45:18.775685 Epoch 1, Training Loss 2.2204538946566372
2022-03-30 16:46:38.553847 Epoch 10, Training Loss 1.4235408211608067
2022-03-30 16:48:07.463176 Epoch 20, Training Loss 1.1908590451378347
2022-03-30 16:49:36.109035 Epoch 30, Training Loss 1.0653670177892651
2022-03-30 16:51:04.259801 Epoch 40, Training Loss 0.9823310786805799
2022-03-30 16:52:35.120001 Epoch 50, Training Loss 0.9267054317552416
2022-03-30 16:54:05.220083 Epoch 60, Training Loss 0.8880417294362012
2022-03-30 16:55:34.247565 Epoch 70, Training Loss 0.8552308700349934
2022-03-30 16:57:03.561531 Epoch 80, Training Loss 0.8270201234866286
2022-03-30 16:58:32.665784 Epoch 90, Training Loss 0.8036015193785548
2022-03-30 17:00:01.756657 Epoch 100, Training Loss 0.783330793918856
2022-03-30 17:01:31.507640 Epoch 110, Training Loss 0.7633491739287706
2022-03-30 17:03:00.325759 Epoch 120, Training Loss 0.7450339214118851
2022-03-30 17:04:29.466556 Epoch 130, Training Loss 0.7305923602007844
2022-03-30 17:05:5

In [13]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                          shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                          shuffle=False)
def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0
        
        with torch.no_grad():
            for imgs, labels in loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.shape[0]
                correct += int((predicted==labels).sum())
                
        print("Accuracy {}: {:.2f}".format(name, correct/total))
        
validate(model, train_loader, val_loader)

Accuracy train: 0.78
Accuracy val: 0.62


In [14]:
class NetDepth(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3,n_chans1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n_chans1,n_chans1 // 2, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(n_chans1 // 2, n_chans1 // 2, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(4 * 4 * n_chans1 // 2, 32)
        self.fc2 = nn.Linear(32, 10)
    
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = F.max_pool2d(torch.tanh(self.conv3(out)), 2)
        out = out.view(-1, 4 * 4 * self.n_chans1 // 2)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [15]:
model = NetDepth()
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 300,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

2022-03-30 17:32:27.345780 Epoch 1, Training Loss 2.3013628827946264
2022-03-30 17:33:53.420263 Epoch 10, Training Loss 1.476427768807277
2022-03-30 17:35:30.875132 Epoch 20, Training Loss 1.188119527095419
2022-03-30 17:37:06.497807 Epoch 30, Training Loss 1.0329284735805238
2022-03-30 17:38:42.209826 Epoch 40, Training Loss 0.9242095260516457
2022-03-30 17:40:20.082832 Epoch 50, Training Loss 0.8534279653178457
2022-03-30 17:41:59.169767 Epoch 60, Training Loss 0.8025574236727127
2022-03-30 17:43:38.601437 Epoch 70, Training Loss 0.7635415945668964
2022-03-30 17:45:17.958841 Epoch 80, Training Loss 0.7308549424419013
2022-03-30 17:46:57.307985 Epoch 90, Training Loss 0.7032619550862276
2022-03-30 17:48:38.141595 Epoch 100, Training Loss 0.6802099860080367
2022-03-30 17:50:16.908770 Epoch 110, Training Loss 0.6603256349292252
2022-03-30 17:51:53.597371 Epoch 120, Training Loss 0.6427061351211479
2022-03-30 17:53:30.300080 Epoch 130, Training Loss 0.6271937866040203
2022-03-30 17:55:06

In [16]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                          shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                          shuffle=False)

validate(model, train_loader, val_loader)

Accuracy train: 0.81
Accuracy val: 0.68


In [ ]:
#problem 2

In [17]:
class ResBlock(nn.Module):
    def __init__(self, n_chans):
        super(ResBlock, self).__init__()
        self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=3,
                            padding=1, bias=False)
        self.batch_norm = nn.BatchNorm2d(num_features=n_chans)
        torch.nn.init.kaiming_normal_(self.conv.weight,
                                nonlinearity='relu')
        torch.nn.init.constant_(self.batch_norm.weight, 0.5)
        torch.nn.init.zeros_(self.batch_norm.bias)

    def forward(self, x):
        out = self.conv(x)
        out = self.batch_norm(out)
        out = torch.relu(out)
        return out + x

In [18]:
class NetResDeep(nn.Module):
    def __init__(self, n_chans1=32, n_blocks=10):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.resblocks = nn.Sequential(
                    *(n_blocks * [ResBlock(n_chans=n_chans1)]))
        self.fc1 = nn.Linear(8 * 8 * n_chans1, 32)
        self.fc2 = nn.Linear(32, 10)
   
    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = self.resblocks(out)
        out = F.max_pool2d(out, 2)
        out = out.view(-1, 8 * 8 * self.n_chans1)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [19]:
model = NetResDeep()
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 300,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

2022-03-30 18:21:18.431655 Epoch 1, Training Loss 1.9942206347080143
2022-03-30 18:23:38.070602 Epoch 10, Training Loss 0.8440433672017149
2022-03-30 18:26:15.286502 Epoch 20, Training Loss 0.5992770129076356
2022-03-30 18:28:53.169631 Epoch 30, Training Loss 0.44179730408865475
2022-03-30 18:31:31.414153 Epoch 40, Training Loss 0.3327786143573806
2022-03-30 18:34:07.274602 Epoch 50, Training Loss 0.27365949068723433
2022-03-30 18:36:43.339798 Epoch 60, Training Loss 0.23048554115054554
2022-03-30 18:39:18.582540 Epoch 70, Training Loss 0.17883013456088045
2022-03-30 18:41:54.895712 Epoch 80, Training Loss 0.16226256753632423
2022-03-30 18:44:30.335085 Epoch 90, Training Loss 0.13333429456354998
2022-03-30 18:47:05.777165 Epoch 100, Training Loss 0.10139390078666227
2022-03-30 18:49:45.232996 Epoch 110, Training Loss 0.11396011390099231
2022-03-30 18:52:25.213393 Epoch 120, Training Loss 0.09061609484824588
2022-03-30 18:55:05.072906 Epoch 130, Training Loss 0.09835557351389047
2022-03

In [20]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                          shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                          shuffle=False)
def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0
        
        with torch.no_grad():
            for imgs, labels in loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.shape[0]
                correct += int((predicted==labels).sum())
                
        print("Accuracy {}: {:.2f}".format(name, correct/total))
        
validate(model, train_loader, val_loader)

Accuracy train: 1.00
Accuracy val: 0.66


In [24]:
def training_loop_l2reg(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)
            
            l2_lambda = 0.001
            l2_norm = sum(p.pow(2.0).sum()
                         for p in model.parameters())
            loss = loss + l2_lambda * l2_norm
            
            optimizer.zero_grad()
            
            loss.backward()
            
            optimizer.step()
            
            loss_train += loss.item()
            
        if epoch == 1 or epoch % 10 ==0:
            print('{} Epoch {}, Training Loss {}'.format(datetime.datetime.now(),
                                                        epoch,
                                                        loss_train / len(train_loader)))

In [25]:
model = NetResDeep()
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop_l2reg(
    n_epochs = 300,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

2022-03-30 19:42:36.305529 Epoch 1, Training Loss 1.9613920851131839
2022-03-30 19:45:07.212596 Epoch 10, Training Loss 0.952520680397063
2022-03-30 19:47:54.860173 Epoch 20, Training Loss 0.7531710693141078
2022-03-30 19:50:44.931584 Epoch 30, Training Loss 0.6405774277189503
2022-03-30 19:53:33.400313 Epoch 40, Training Loss 0.5668739221056404
2022-03-30 19:56:21.793212 Epoch 50, Training Loss 0.5318197994814504
2022-03-30 19:59:10.588577 Epoch 60, Training Loss 0.506883549835066
2022-03-30 20:01:58.880331 Epoch 70, Training Loss 0.48401006480769426
2022-03-30 20:04:46.583662 Epoch 80, Training Loss 0.48376262847267454
2022-03-30 20:07:35.313551 Epoch 90, Training Loss 0.4636950268388709
2022-03-30 20:10:24.326739 Epoch 100, Training Loss 0.4620126979735196
2022-03-30 20:13:13.363692 Epoch 110, Training Loss 0.43894982391306203
2022-03-30 20:16:02.578973 Epoch 120, Training Loss 0.45495226845869324
2022-03-30 20:18:51.940689 Epoch 130, Training Loss 0.4355231932819347
2022-03-30 20:2

In [26]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                          shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                          shuffle=False)

validate(model, train_loader, val_loader)

Accuracy train: 0.92
Accuracy val: 0.66


In [ ]:
class NetDropout(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3,n_chans1, kernel_size=3, padding=1)
        self.conv1_dropout = nn.Dropout2d(p=0.4)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3, padding=1)
        self.conv2_dropout = nn.Dropout2d(p=0.4)
        self.fc1 = nn.Linear(8 * 8 * n_chans // 2, 32)
        self.fc2 = nn.Linear(32, 10)
    
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = self.conv1_dropout(out)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = self.conv2_dropout(out)
        out = out.view(-1, 8 * 8 * n_chans // 2)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [ ]:
model = NetResDeep()
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop_l2reg(
    n_epochs = 300,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

In [ ]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                          shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                          shuffle=False)

validate(model, train_loader, val_loader)

In [ ]:
class NetBatchNorm(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3,n_chans1, kernel_size=3, padding=1)
        self.conv1_dropout = nn.Dropout2d(p=0.4)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3, padding=1)
        self.conv2_dropout = nn.Dropout2d(p=0.4)
        self.fc1 = nn.Linear(8 * 8 * n_chans // 2, 32)
        self.fc2 = nn.Linear(32, 10)
    
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = self.conv1_dropout(out)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = self.conv2_dropout(out)
        out = out.view(-1, 8 * 8 * n_chans // 2)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [ ]:
model = NetResDeep()
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop_l2reg(
    n_epochs = 300,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)